### Packages
***

In [1]:
library(igraph)
library(resample)
library(textTinyR)
library(Matrix)
library(pracma)


載入套件：‘igraph’


下列物件被遮斷自 ‘package:stats’:

    decompose, spectrum


下列物件被遮斷自 ‘package:base’:

    union


載入需要的套件：Matrix


載入套件：‘pracma’


下列物件被遮斷自 ‘package:Matrix’:

    expm, lu, tril, triu




### 3. PageRank: The PageRank algorithm, as used by the Google search engine, exploits the linkage struc- ture of the web to compute global “importance” scores that can be used to influence the ranking of search results. Here, we use random walk to simulate PageRank.
***

#### (a) We are going to create a directed random network with 900 nodes, using the preferen- tial attachment model. Note that in a directed preferential attachment network, the out-degree of every node is m, while the in-degrees follow a power law distribution. One problem of performing random walk in such a network is that, the very first node will have no outbounding edges, and be a “black hole” which a random walker can never “escape” from. To address that, let’s generate another 900-node random network with preferential attachment model, and merge the two networks by adding the edges of the second graph to the first graph with a shuffling of the indices of the nodes. See below figure for example. Create such a network using m = 4. Measure the probability that the walker visits each node. Is this probability related to the degree of the nodes?

<img src="f1.png" width="500">

> Ans: 

#### (b) In all previous questions, we didn’t have any teleportation. Now, we use a teleporta- tion probability of α = 0.2 (teleport out of a node with prob=0.2 instead of going to its neighbor). By performing random walks on the network created in 3(a), measure the probability that the walker visits each node. How is this probability related to the degree of the node and α ?

> Ans: 

### 4. Personalized PageRank: While the use of PageRank has proven very effective, the web’s rapid growth in size and diversity drives an increasing demand for greater flexibility in ranking. Ideally, each user should be able to define their own notion of importance for each individual query.
***

#### (a) Suppose you have your own notion of importance. Your interest in a node is propor- tional to the node’s PageRank, because you totally rely upon Google to decide which website to visit (assume that these nodes represent websites). Again, use random walk on network generated in question 3 to simulate this personalized PageRank. Here the teleportation probability to each node is proportional to its PageRank (as opposed to the regular PageRank, where at teleportation, the chance of visiting all nodes are the same and equal to 1/N ). Again, let the teleportation probability be equal to α = 0.2. Compare the results with 3(a).

> Ans: 

#### (b) Find two nodes in the network with median PageRanks. Repeat part 4(a) if tele- portations land only on those two nodes (with probabilities 1/2, 1/2). How are the PageRank values affected?

> Ans: 

#### (c) More or less, 4(b) is what happens in the real world, in that a user browsing the web only teleports to a set of trusted web pages. However, this is against the assumption of normal PageRank, where we assume that people’s interest in all nodes are the same. Can you take into account the effect of this self-reinforcement and adjust the PageRank equation?

> Ans: 